<a href="https://colab.research.google.com/github/sdenyskov/sd995_ads_2024/blob/main/notebooks/02-access-assess-geospatial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical 2

### [Radzim Sendyka](https://www.cst.cam.ac.uk/people/rs2071), University

of Cambridge

### [Christian Cabrera](https://www.cst.cam.ac.uk/people/chc79), University

of Cambridge

### [Carl Henrik Ek](http://carlhenrik.com), University of Cambridge

### [Neil D. Lawrence](http://inverseprobability.com), University of

Cambridge

### 2024-11-07

**Abstract**: In this lab session we look at working with geospatial
data, in conjunction with the house prices dataset you created in the
previous practicals.

$$
$$

::: {.cell .markdown}

<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!---->
<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!-- The last names to be defined. Should be defined entirely in terms of macros from above-->
<!--

-->

**The check Session for this Practical is 12th November 2024.**
Prerequisite: practical 1, and a working database with tables price paid
data (i.e., `pp_data`) and postcodes(i.e., `postcode_data`)

In this lab session we look at working with geospacial data, in
conjunction with the house prices dataset you created in the previous
practicals. The goal is to enrich the data from the first practical with
geographic data enabling better informed data analysis. Access to the
price paid database is needed to complete some of the below exercises.
You are asked to write reusable code that will help you in the
assessment.

## Accessing Open Street Maps

<span class="editsection-bracket"
style="">\[</span><span class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_datasets/includes/accessing-osm.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_datasets/includes/accessing-osm.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

[Open Street Maps
(OSM)](https://www.openstreetmap.org/#map=6/54.91/-3.43) is an open
geographic database that can provide useful information about different
locations and places in the planet. In this example, we will download
data about the city of Kampala, Uganda. As always, we should start by
installing some Python packages.

In [ ]:
%pip install osmnx
%pip uninstall --yes matplotlib
%pip install matplotlib==3.7.1

In [463]:
import osmnx as ox
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='osmnx')

We will download data of Kamplala, Uganda, which has the following
latitude and longitude.

In [464]:
place_name = "Kampala, Uganda"

latitude = 0.347596 # Kampala latitude
longitude = 32.582520 # Kampala longitude

In [465]:
placestub = place_name.lower().replace(' ', '-').replace(',','')

We’ll create a bounding box which is 0.02 degrees wide, 1 degree is
around 111km ([circumference of the Earth is around 40,000
km](https://en.wikipedia.org/wiki/Metre) and 40,000/360=111km). Note:
will this approximation work well in all countries?

In [466]:
box_width = 0.02 # About 2.2 km
box_height = 0.02
north = latitude + box_height/2
south = latitude - box_width/2
west = longitude - box_width/2
east = longitude + box_width/2

Now we’ll download a set of points of interest from OpenStreetMap. We
can specify the points of interest we’re interested in by building a
small dictionary containing their labels as follows. A Point of Interest
is a location with certain importance in the geographic area. They can
vary from amenities to touristic places as you can see in the following.

In [467]:
# Retrieve POIs
tags = {
    "amenity": True,
    "buildings": True,
    "historic": True,
    "leisure": True,
    "shop": True,
    "tourism": True,
    "religion": True,
    "memorial": True
}

We can use `osmnx` to download all such points of interest within a
given bounding box.

In [ ]:
pois = ox.geometries_from_bbox(north, south, east, west, tags)

That operation can take some time, particularly as the bounding box
grows larger. Once it is complete we can check how many points of
interest we have found.

In [ ]:
print("There are {number} points of interest surrounding {placename} latitude: {latitude}, longitude: {longitude}".format(number=len(pois), placename=place_name, latitude=latitude, longitude=longitude))

And then we can examine their contents in more detail.

In [ ]:
pois

### We notice a few things:

1.  Points of interest do not have a consistent OpenStreetMap
    `element_type`, some are `node`, others are `relation` and we also
    have `way`. You can find out more about elements in OpenStreetMap on
    [this wiki page](https://wiki.openstreetmap.org/wiki/Elements). This
    will become important when tidying up the data for next stage
    processing.

2.  Many of the values are missing. In SQL we would express a missing
    value as `NULL`. But in `pandas` a missing value is expressed as
    not-a-number, `NaN`. This is quite a common standard, but it is not
    the only standard. Sometimes data is collected and coded with an
    “unreasonable” value for a missing value. For example, someone might
    set missing values for heights to -999. The concept is that this is
    an obviously void “height” and would trigger a human user to check
    whether it’s a missing value. Of course, this is obvious to humans,
    but not necessarily to a computer!

Nodes, ways and relations in OpenStreetMap all have different *keys*
associated with them. The data is not structured in standard database
columns. Different points of interest might have different keys present
or absent. We might be interested in the following keys.

In [471]:
keys = ["name",
        "addr:city",
        "addr:postcode",
        "amenity",
        "building",
        "building:name",
        "building:colour",
        "building:material",
        "historic",
        "memorial",
        "religion",
        "tourism",
        "emergency",
        "leisure",
        "shop"]

But our downloaded `gdf` may have fewer keys.

In [ ]:
pois.columns.values

We can write a short piece of code to discover which keys are missing
drom the data frame’s columns.

In [ ]:
for key in keys:
    if key not in pois.columns:
        print(key)

present_keys = [key for key in keys if key in pois.columns]
pois[present_keys]

This gives us the relevant points of interest (part of the map). If we’d
like to see the entire street network, we can download the entire graph
from the location.

In [ ]:
graph = ox.graph_from_bbox(north, south, east, west)

# Retrieve nodes and edges
nodes, edges = ox.graph_to_gdfs(graph)

# Get place boundary related to the place name as a geodataframe
area = ox.geocode_to_gdf(place_name)

Which we can then render as follows.

In [ ]:
import matplotlib.pyplot as plt

%pip install --upgrade matplotlib

In [ ]:
fig, ax = plt.subplots()

# Plot the footprint
area.plot(ax=ax, facecolor="white")

# Plot street edges
edges.plot(ax=ax, linewidth=1, edgecolor="dimgray")

ax.set_xlim([west, east])
ax.set_ylim([south, north])
ax.set_xlabel("longitude")
ax.set_ylabel("latitude")

# Plot all POIs
pois.plot(ax=ax, color="blue", alpha=0.7, markersize=10)
plt.tight_layout()

In [ ]:
# Plot a subset of the POIs (e.g., tourist places)
# Create figure
fig, ax = plt.subplots()

# Plot the footprint
area.plot(ax=ax, facecolor="white")

# Plot street edges
edges.plot(ax=ax, linewidth=1, edgecolor="dimgray")

ax.set_xlim([west, east])
ax.set_ylim([south, north])
ax.set_xlabel("longitude")
ax.set_ylabel("latitude")

# Plot tourist places
tourist_places = pois[pois.tourism.notnull()]
tourist_places.plot(ax=ax, color="blue", alpha=1, markersize=50)
plt.tight_layout()

We have the POI information on all tourist places structured in a
geodataframe. To work with them in a machine learning algorithm, it will
be easier to convert them to a pandas DataFrame.

In [478]:
import pandas as pd

In [ ]:
pois_df = pd.DataFrame(pois)
pois_df['latitude'] = pois_df.apply(lambda row: row.geometry.centroid.y, axis=1)
pois_df['longitude'] = pois_df.apply(lambda row: row.geometry.centroid.x, axis=1)

tourist_places_df = pois_df[pois_df.tourism.notnull()]
print(len(tourist_places_df))
tourist_places_df

In [ ]:
poi_counts = {}

poi_types =["amenity", "historic", "leisure", "shop", "tourism", "religion", "memorial"]

for tag in poi_types:
  if tag in pois_df.columns:
    poi_counts[tag] = pois_df[tag].notnull().sum()
  else:
    poi_counts[tag] = 0

poi_counts_df = pd.DataFrame(list(poi_counts.items()), columns=['POI Type', 'Count'])



poi_counts_df

## Assessing the Available OpenStreetMap Features

<span class="editsection-bracket"
style="">\[</span><span class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_datasets/includes/assessing-osm.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_datasets/includes/assessing-osm.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

In the course assessment you will be given the task of constructing a
prediction system for various indicators at a given location. We expect
that knowledge of the local region around the property should be helpful
in making those predictions. To evaluate this we will now look at
[OpenStreetMap](https://www.openstreetmap.org) as a data source.

In this section, you should follow the methodology used in the above
example to extract summary OSM information that can be useful in making
predictions about an area. Use code from the example to construct a
function that summarises the number of various points of interest in a
target area. You should write reusable code that allows you to explore
the characteristics of different points of interest.

In [481]:
def count_pois_near_coordinates(latitude: float, longitude: float, tags: dict, distance_km: float = 1.0) -> dict:
    """
    Count Points of Interest (POIs) near a given pair of coordinates within a specified distance.
    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        tags (dict): A dictionary of OSM tags to filter the POIs (e.g., {'amenity': True, 'tourism': True}).
        distance_km (float): The distance around the location in kilometers. Default is 1 km.
    Returns:
        dict: A dictionary where keys are the OSM tags and values are the counts of POIs for each tag.
    """
    
    distance = distance_km / 111
    north = latitude + distance
    south = latitude - distance
    west = longitude - distance
    east = longitude + distance

    pois = ox.geometries_from_bbox(north, south, east, west, tags)
    pois_df = pd.DataFrame(pois)
    
    poi_counts = {}
    for tag, value in tags.items():
        if tag in pois_df.columns:
            if value is True:
                # count all POIs for this tag
                poi_counts[tag] = pois_df[tag].notnull().sum()
            elif isinstance(value, list):
                # count POIs that match one of the list values
                poi_counts[tag] = pois_df[tag].isin(value).sum()
            else:
                # raise an error
                raise ValueError(f"Unexpected value: tags[{tag}] = {value}")
        else:
            poi_counts[tag] = 0
    
    return poi_counts

Now that you have written reusable code, choose the tags you want to
query. This should be different from the tags used in the example. You
can also search for specific tags like this:
`"amenity": ["university", ...`.

In [482]:
tags = {
    "amenity": True,
    "historic": True,
    "leisure": True,
    "shop": True,
    "sport": ["tennis", "soccer", "squash", "multi", "bowls"],
    "tourism": True,
    "healthcare": True,
}

Here there are 13 UK locations.

In [483]:
locations_dict = {
    "Cambridge": (52.2054, 0.1132),
    "Oxford": (51.7570, -1.2545),
    "Euston Square": (51.5246, -0.1340),
    "Temple": (51.5115, -0.1160),
    "Kensington": (51.4988, -0.1749),
    "Barnsley": (53.5526, -1.4797),
    "Mansfield": (53.1472, -1.1987),
    "Wakefield": (53.6848, -1.5039),
    "Sunderland": (54.9069, -1.3838),
    "Rotherham": (53.4300, -1.3568),
    "Doncaster": (53.5228, -1.1288),
    "Chesterfield": (53.2350, -1.4210),
    "Huddersfield": (53.6450, -1.7794)
    }

### Exercise 1

Use your code to query the OSM feature counts for each of them, and
combine them into one dataframe.

### Exercise 1 Answer

Write your answer to Exercise 1 here

In [ ]:
osm_feature_counts = {header: [] for header in ["location"] + list(tags.keys())}

for location, coords in locations_dict.items():

    latitude, longitude = coords[0], coords[1]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        poi_counts = count_pois_near_coordinates(latitude, longitude, tags)

    osm_feature_counts["location"].append(location)
    for tag, poi_count in poi_counts.items():
        osm_feature_counts[tag].append(poi_count)
    
    print(f"Location {location} has been processed.")

osm_feature_counts_df = pd.DataFrame(osm_feature_counts)

In [ ]:
osm_feature_counts_df

### Exercise 2

Use k-means clustering or another clustering method to try to find
clusters of similar areas, based on nearby OSM features.

### Exercise 2 Answer

Write your answer to Exercise 2 here

In [ ]:
%pip install scikit-learn

In [530]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [531]:
def kmeans_clusters(data, n_clusters, normalise):

    data = data.drop(columns=['location'])

    if normalise:
        scaler = MinMaxScaler()
        scaled_data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
    else:
        scaled_data = data
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(scaled_data)
    clusters = kmeans.labels_

    return clusters

In [532]:
def extend_feature_df(data, locations_dict, clusters):
    
    latitudes = [locations_dict[location][0] for location in locations_dict]
    longitudes = [locations_dict[location][1] for location in locations_dict]

    to_concat = pd.DataFrame({"latitude": latitudes, "longitude": longitudes, "cluster": clusters})
    df = pd.concat([data, to_concat], axis=1)

    return df

In [534]:
clusters = kmeans_clusters(data = osm_feature_counts_df.copy(), n_clusters = 3, normalise = False)
osm_feature_counts_df_clus = extend_feature_df(data = osm_feature_counts_df.copy(), locations_dict = locations_dict, clusters = clusters)

clusters = kmeans_clusters(data = osm_feature_counts_df.copy(), n_clusters = 3, normalise = True)
osm_feature_counts_df_clus_norm = extend_feature_df(data = osm_feature_counts_df.copy(), locations_dict = locations_dict, clusters = clusters)

In [ ]:
osm_feature_counts_df_clus.sort_values(by='cluster')

In [ ]:
osm_feature_counts_df_clus_norm.sort_values(by='cluster')

### Exercise 3

Investigate the locations yourself, and assign them categories based on
your interpretation. Visualise and compare your manual assignments
against your clustering results.

### Exercise 3 Answer

Write your answer to Exercise 3 here

In [ ]:
"""
In order to do that, let's check the dataframe with feature counts
"""

osm_feature_counts_df

In [537]:
clusters = [2, 2, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0]

osm_feature_counts_df_clus_manu = extend_feature_df(data = osm_feature_counts_df.copy(), locations_dict = locations_dict, clusters = clusters)

In [ ]:
osm_feature_counts_df_clus_manu.sort_values(by='cluster')

In [939]:
def plot_clusters(clustered_locations):
    cluster_colors = {0: 'red', 1: 'green', 2: 'blue'}

    plt.figure(figsize = (10, 10))
    for cluster, color in cluster_colors.items():
        cluster_data = clustered_locations[clustered_locations['cluster'] == cluster]
        plt.scatter(
            cluster_data['longitude'], 
            cluster_data['latitude'], 
            label=f'Cluster {cluster}', 
            color=color
        )
        for _, row in cluster_data.iterrows():
            plt.text(row['longitude'] + 0.02, row['latitude'] + 0.02, row['location'])

    plt.axis('equal')
    plt.grid(True)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend(title='Clusters')
    plt.title('Map of cities by cluster')
    plt.show()

In [ ]:
plot_clusters(osm_feature_counts_df_clus)
plot_clusters(osm_feature_counts_df_clus_manu)
plot_clusters(osm_feature_counts_df_clus_norm)

### Exercise 4

Normalise your dataframe and compute a distance matrix for the
locations. Visualise it, and compare the outcode with your previous
clustering results.

### Exercise 4 Answer

Write your answer to Exercise 4 here

In [ ]:
%pip install numpy

In [543]:
import numpy as np

In [546]:
osm_feature_counts_df_coor = osm_feature_counts_df_clus.copy().drop(columns=['cluster'])

In [ ]:
osm_feature_counts_df_coor

In [551]:
def distance(lat1, lon1, lat2, lon2):
    """
    Calculates distance between two points with given coordinates using 111 km approximation
    """
    return np.sqrt((lat1 - lat2) * 111 * (lat1 - lat2) * 111 + (lon1 - lon2) * 111 * (lon1 - lon2) * 111)

In [ ]:
# Create distance matrix filled with 0s
dim = len(osm_feature_counts_df_coor)
distance_matrix = np.zeros((dim, dim))

# Calculate distance matrix
for i in range(dim):
    for j in range(dim):
        distance_matrix[i, j] = distance(
            osm_feature_counts_df_coor.loc[i, 'latitude'], 
            osm_feature_counts_df_coor.loc[i, 'longitude'],
            osm_feature_counts_df_coor.loc[j, 'latitude'], 
            osm_feature_counts_df_coor.loc[j, 'longitude']
        )

# Convert distance matrix to DataFrame
distance_matrix_df = pd.DataFrame(distance_matrix,
                                  columns=osm_feature_counts_df_coor['location'],
                                  index=osm_feature_counts_df_coor['location'])

# Plot distance matrix
plt.figure(figsize=(10, 8))
plt.imshow(distance_matrix_df, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Distance in km')

plt.title("Distance matrix")
plt.xticks(np.arange(dim), osm_feature_counts_df_coor['location'], rotation=45, ha='right')
plt.yticks(np.arange(dim), osm_feature_counts_df_coor['location'])
plt.tight_layout()
plt.show()


In [557]:

def normalise_df(data):

    locations = data['location']
    data = data.drop(columns=['location'])
    
    scaler = MinMaxScaler()
    scaled_data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

    locations_df = pd.DataFrame({"location": locations})
    normalised_df = pd.concat([locations_df, scaled_data], axis=1)

    return normalised_df

In [558]:
osm_feature_counts_df_norm = normalise_df(osm_feature_counts_df.copy())

In [ ]:
osm_feature_counts_df_norm

### Exercise 5

Which features you included were correlated among each other?
Investigate and plot a feature correlation matrix. What do these results
say about your feature selection?

### Exercise 5 Answer

Write your answer to Exercise 5 here

In [ ]:
df = osm_feature_counts_df_norm.copy().drop(columns=['location'])
correlation_matrix = df.corr()
dim = len(correlation_matrix.columns)

# Plot correlation matrix
plt.figure(figsize=(10, 8))
plt.imshow(correlation_matrix, cmap='coolwarm', interpolation='nearest')
plt.colorbar(label='Correlation Coefficient')
plt.title("Correlation Matrix of Features")
plt.xticks(range(dim), correlation_matrix.columns, rotation=45, ha='right')
plt.yticks(range(dim), correlation_matrix.columns)

# Add correlation values to cells
for i in range(dim):
    for j in range(dim):
        plt.text(j, i, f"{correlation_matrix.iloc[i, j]:.2f}", ha="center", va="center", color="black")

plt.tight_layout()
plt.show()

## Joining Spatial Data

<span class="editsection-bracket"
style="">\[</span><span class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_access/includes/spatial-join.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_access/includes/spatial-join.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

### Matching OpenStreetMap and House Prices data

In this exercise you will download the geographies of houses from
OpenStreetMap and map them to visualise the records you see in the house
price dataset. This is a data linking and validation exercise.

The latitude and longitude of Cambridge are as follows:

In [574]:
place_name = "Cambridge"
latitude = 52.1951
longitude = 0.1313

We want to execute an SQL query on your database to select all houses in
a 1km x 1km region around the centre of Cambridge that have been part of
housing transactions since 2020.

This operation can take a very long time. This is because the table is
not indexed on coordinate data, and therefore the query has to check
tens of millions of rows. This can be fixed by constructing an index on
the `latitude` and `longitude` values, using `BTREE` to make a joint
index. *Note that indexing can take a long time.* Consider also indexing
your table by other variables you might find useful later.

### Exercise 6

Index the table on the coordinate data using a `BTREE` and index other
columns you might find useful.

### Exercise 6 Answer

#### Installs and imports

In [ ]:
%pip install pyyaml
%pip install ipython-sql
%pip install PyMySQL
%pip install pymysql
%load_ext sql

In [576]:
import yaml
import pymysql

In [ ]:
%pip uninstall --yes fynesse
%pip install git+https://github.com/sdenyskov/sd995_ads_2024.git

In [578]:
import fynesse

#### Connection and cursor initiation

In [579]:
with open("credentials.yaml") as file:
  credentials = yaml.safe_load(file)
username = credentials["username"]
password = credentials["password"]
url = credentials["url"]
port = credentials["port"]

In [697]:
conn = fynesse.access.create_connection(user=username, password=password, host=url, database='ads_2024')
cursor = conn.cursor()

Connection established!


#### Uploading data

In [ ]:
for year in range(2020, 2023):
    fynesse.access.housing_upload_join_data(conn, year)

#### Indexing tables

In [ ]:
cursor.execute("CREATE INDEX idx_lat_lon_btree ON `prices_coordinates_data` (latitude, longitude)")
conn.commit()

#### Checking properties of tables

In [ ]:
cursor.execute("SHOW INDEXES FROM `prices_coordinates_data`")
results = cursor.fetchall()
for result in results:
    print(result)

In [ ]:
cursor.execute("SELECT MIN(date_of_transfer) AS min_value, MAX(date_of_transfer) AS max_value FROM `prices_coordinates_data`")
results = cursor.fetchall()
print(results)

In [ ]:
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'prices_coordinates_data'")
results = cursor.fetchall()
for result in results:
    print(result[0])

In [ ]:
cursor.execute("SELECT * FROM `prices_coordinates_data` LIMIT 5")
results = cursor.fetchall()
for result in results:
    print(result)

### Exercise 7

Write an SQL query on your database to select all houses in a 1km x 1km
region around the centre of Cambridge that have been part of housing
transactions since 2020.

### Exercise 7 Answer

#### Calculating bounds of the area

In [768]:
def get_bounds(latitude, longitude, box_dim_km):

    box_dim = box_dim_km / 111
    north = latitude + box_dim / 2
    south = latitude - box_dim / 2
    west = longitude - box_dim / 2
    east = longitude + box_dim / 2

    return (north, south, west, east)

In [769]:
bounds = get_bounds(latitude, longitude, 1)
north, south, west, east = bounds

#### Forming all_houses_from_db

In [822]:
def get_houses_in_the_area(cursor, bounds):

    north, south, west, east = bounds
    columns = ["price", 
               "date_of_transfer", 
               "postcode", 
               "property_type", 
               "new_build_flag", 
               "tenure_type", 
               "locality", 
               "town_city", 
               "district", 
               "county", 
               "country", 
               "latitude", 
               "longitude", 
               "db_id", 
               "primary_addressable_object_name", 
               "secondary_addressable_object_name", 
               "street"]

    # cursor.execute(f"SELECT * FROM `prices_coordinates_data` WHERE latitude BETWEEN {south} AND {north} AND longitude BETWEEN {west} AND {east} AND date_of_transfer >= '2020-01-01'")
    cursor.execute(f"SELECT pcd.*, pp.primary_addressable_object_name, pp.secondary_addressable_object_name, pp.street FROM prices_coordinates_data AS pcd JOIN pp_data AS pp ON pcd.postcode = pp.postcode AND pcd.date_of_transfer = pp.date_of_transfer AND pcd.price = pp.price WHERE pcd.latitude BETWEEN {south} AND {north} AND pcd.longitude BETWEEN {west} AND {east} AND pcd.date_of_transfer >= '2020-01-01'")
    results = cursor.fetchall()

    df = pd.DataFrame(results, columns = columns)

    return df


In [ ]:
all_houses_from_db = get_houses_in_the_area(cursor, bounds)
print(len(all_houses_from_db))

all_houses_from_db = all_houses_from_db.drop_duplicates(subset=all_houses_from_db.columns.difference(['db_id']))
print(len(all_houses_from_db))

In [ ]:
all_houses_from_db

### Exercise 8

Get information about all the buildings in that area from OpenStreetMaps
(`'building': True`). You will need their address information
(`addr:housenumber`, `addr:street`, `addr:postcode`, …) and geometry
polygon (`geometries_from_bbox`). Construct a dataframe that lists all
OSM buildings in the area that have a full address, along with their
area (in square meters). Plot a map of the area, using color to mark the
buildings with addresses and the ones without.

### Exercise 8 Answer

#### Installs and imports

In [ ]:
%pip install geopandas

In [711]:
import geopandas as gpd

#### Forming all_buildings_from_osm

In [ ]:
# Get information about all buildings in the area
all_buildings_from_osm = ox.geometries_from_bbox(north, south, east, west, tags = {'building': True})

# Filter polygons only
all_buildings_from_osm = all_buildings_from_osm[all_buildings_from_osm['geometry'].geom_type == 'Polygon']

# From geographical coordinate reference system to projected reference system, then compute area in square meters
all_buildings_from_osm['area_m2'] = all_buildings_from_osm.copy().to_crs(epsg=3395).geometry.area

print(len(all_buildings_from_osm))

all_buildings_from_osm

#### Forming valid_buildings_from_osm

In [ ]:
# Filter buildings based on the presence of address
valid_buildings_from_osm = all_buildings_from_osm[all_buildings_from_osm['addr:street'].notnull() 
                                                  & all_buildings_from_osm['addr:housenumber'].notnull() 
                                                  & all_buildings_from_osm['addr:postcode'].notnull()
                                                  & all_buildings_from_osm['addr:city'].notnull()]

print(len(valid_buildings_from_osm))

valid_buildings_from_osm

#### Plotting the area

In [ ]:
# Retrieve main graph
graph = ox.graph_from_bbox(north, south, east, west)

# Retrieve nodes and edges
nodes, edges = ox.graph_to_gdfs(graph)

# Get place boundary related to the place name as a geodataframe
area = ox.geocode_to_gdf(place_name)

# Initialise plot
fig, ax = plt.subplots()

# Plot the footprint
area.plot(ax=ax, facecolor="white")

# Plot street edges
edges.plot(ax=ax, linewidth=1, edgecolor="dimgray")

ax.set_xlim([west, east])
ax.set_ylim([south, north])
ax.set_xlabel("longitude")
ax.set_ylabel("latitude")

# Plot buildings
all_buildings_from_osm.plot(ax=ax, color="grey", alpha=0.7, markersize=10)
valid_buildings_from_osm.plot(ax=ax, color="blue", alpha=0.7, markersize=10)
plt.tight_layout()

### Exercise 9

Match the houses you found in the price paid dataset with the buildings
on OpenStreetMaps based on their addresses. Can this be applied to all
building types? Are there any PP transactions which you couldn’t match
to an OSM building, or any OSM buildings you coulnd’t match to a PP
transaction? If so, what could be the reason for this? Do you employ any
techniques to find non-exact matches? If yes, what matches would you
have missed without it? Are you encountering false positive matches? Use
this address matching to merge the two dataframes.

### Exercise 9 Answer

Write your answer to Exercise 9 here

In [ ]:
all_houses_from_db[['price', 
                    'date_of_transfer', 
                    'postcode', 
                    'town_city', 
                    'latitude', 
                    'longitude', 
                    'primary_addressable_object_name', 
                    'secondary_addressable_object_name', 
                    'street', 
                    'db_id']][all_houses_from_db['street'] == 'STATION ROAD']

In [ ]:
valid_buildings_from_osm[['geometry', 
                          'addr:city', 
                          'addr:housenumber', 
                          'addr:postcode', 
                          'addr:street', 
                          'name', 
                          'area_m2']][valid_buildings_from_osm['addr:street'] == 'Station Road']

In [937]:
def capitalize(x):
    return x.upper() if isinstance(x, str) else str(x)

In [938]:
# Create a copy of valid_buildings_from_osm
matched_buildings_from_osm = valid_buildings_from_osm.copy()

# Create empty lists to store matched data
matched_price = []
matched_date_of_transfer = []
matched_latitude = []
matched_longitude = []

# Iterate through the rows of matched_buildings_from_osm
for _, osm_row in matched_buildings_from_osm.iterrows():
    name = capitalize(osm_row['name'])
    housenumber = capitalize(osm_row['addr:housenumber'])
    street = capitalize(osm_row['addr:street'])
    city = capitalize(osm_row['addr:city'])
    postcode = capitalize(osm_row['addr:postcode'])

    # Iterate through the rows of all_houses_from_db
    match_found = False
    for _, db_row in all_houses_from_db.iterrows():
        primary_addressable_object_name = capitalize(db_row['primary_addressable_object_name'])
        secondary_addressable_object_name = capitalize(db_row['secondary_addressable_object_name'])
        db_street = capitalize(db_row['street'])
        db_city = capitalize(db_row['town_city'])
        db_postcode = capitalize(db_row['postcode'])
        
        # Check if street, postcode, and city match
        if street == db_street and postcode == db_postcode and city == db_city:
            if (housenumber == primary_addressable_object_name) or (housenumber == secondary_addressable_object_name) or (name in primary_addressable_object_name) or (name in secondary_addressable_object_name):
                matched_price.append(db_row['price'])
                matched_date_of_transfer.append(db_row['date_of_transfer'])
                matched_latitude.append(db_row['latitude'])
                matched_longitude.append(db_row['longitude'])
                match_found = True
                break

    # If no match was found, append None to the lists
    if not match_found:
        matched_price.append(None)
        matched_date_of_transfer.append(None)
        matched_latitude.append(None)
        matched_longitude.append(None)

# Add the matched data as new columns in matched_buildings_from_osm
matched_buildings_from_osm['price'] = matched_price
matched_buildings_from_osm['date_of_transfer'] = matched_date_of_transfer
matched_buildings_from_osm['latitude'] = matched_latitude
matched_buildings_from_osm['longitude'] = matched_longitude

# Remove rows where no match was found
matched_buildings_from_osm.dropna(subset=['price', 'date_of_transfer', 'latitude', 'longitude'], inplace=True)

In [ ]:
matched_buildings_from_osm[['addr:city', 
                            'addr:housenumber', 
                            'addr:postcode', 
                            'addr:street', 
                            'name', 
                            'area_m2',
                            'price', 
                            'date_of_transfer', 
                            'latitude', 
                            'longitude']]

In [ ]:
print(matched_buildings_from_osm.columns)

In [ ]:
# Retrieve main graph
graph = ox.graph_from_bbox(north, south, east, west)

# Retrieve nodes and edges
nodes, edges = ox.graph_to_gdfs(graph)

# Get place boundary related to the place name as a geodataframe
area = ox.geocode_to_gdf(place_name)

# Initialise plot
fig, ax = plt.subplots()

# Plot the footprint
area.plot(ax=ax, facecolor="white")

# Plot street edges
edges.plot(ax=ax, linewidth=1, edgecolor="dimgray")

ax.set_xlim([west, east])
ax.set_ylim([south, north])
ax.set_xlabel("longitude")
ax.set_ylabel("latitude")

# Plot buildings
all_buildings_from_osm.plot(ax=ax, color="grey", alpha=0.7, markersize=10)
valid_buildings_from_osm.plot(ax=ax, color="blue", alpha=0.7, markersize=10)
matched_buildings_from_osm.plot(ax=ax, color="red", alpha=0.7, markersize=10)
plt.tight_layout()

### Exercise 10

Examine the relationship between the price and area of a property. -
What other variables do you need to account for? - Is the correlation as
strong as you would expect? - What factors could be impacting this?

Visualise the relationships you found.

### Exercise 10 Answer

Write your answer to Exercise 10 here

In [ ]:
df = matched_buildings_from_osm.sort_values(by='date_of_transfer')
df[['addr:city', 
    'addr:housenumber', 
    'addr:postcode', 
    'addr:street', 
    'name', 
    'area_m2',
    'price', 
    'date_of_transfer', 
    'latitude', 
    'longitude']]

In [ ]:
corr = df["price"].corr(df["area_m2"])
corr

In [ ]:
dates = df['date_of_transfer']
price_per_sqm = df["price"] / df["area_m2"]
print(price_per_sqm)

plt.figure(figsize=(10, 6))
plt.plot(dates, price_per_sqm, marker='o', linestyle='', color='b')

plt.xlabel('Date')
plt.ylabel('Price per sqm')
plt.title('Price per sqm over time')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [935]:
# List of declared DataFrames:
# osm_feature_counts_df
# osm_feature_counts_df_clus
# osm_feature_counts_df_clus_norm
# osm_feature_counts_df_clus_manu
# osm_feature_counts_df_coor
# osm_feature_counts_df_norm
# all_houses_from_db
# all_buildings_from_osm
# valid_buildings_from_osm
# matched_buildings_from_osm

Demonstrate the reusability of your code by executing the same analysis
for Oxford.

In [936]:
place_name = "Oxford"

latitude = 51.7520
longitude = -1.2577

### Exercise 11

Replicating the same analysis for Oxford. You do not need to answer all
the questions again, but you should show that your code works for this
new input without the need to modify it. You should use the Fynesse
library for this. Finish by plotting a map of the area and the
correlation you find.

### Exercise 11 Answer

Write your answer to Exercise 11 here

In [ ]:
# Use this box for any code you need



## Conclusions

You should find some of the code you wrote above useful in your final
assessment. Make sure you wrote the code to be reusable and efficient,
and do include it in your Fynesse library. The functions you are
particularly likely to reuse are the OSM feature search, and map
visualisation functions.

### Exercise 12

Add relevant code to your Fynesse library. Demonstrate this was
successful by installing your library below and calling at least two
example functions.

### Exercise 12 Answer

Write your answer to Exercise 12 here

In [ ]:
# Use this box for any code you need



## Thanks!

For more information on these subjects and more you might want to check
the following resources.

-   book: [The Atomic
    Human](https://www.penguin.co.uk/books/455130/the-atomic-human-by-lawrence-neil-d/9780241625248)
-   twitter: [@lawrennd](https://twitter.com/lawrennd)
-   podcast: [The Talking Machines](http://thetalkingmachines.com)
-   newspaper: [Guardian Profile
    Page](http://www.theguardian.com/profile/neil-lawrence)
-   blog:
    [http://inverseprobability.com](http://inverseprobability.com/blog.html)

## References